In [1]:
import pandas as pd
import urllib
import os
import httplib2
import numpy as np
from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
import sys

sys.path.append('../src/data/')
import _init_paths
import imagedownloader
import pref_utils
from subprocess import Popen, PIPE
from os import environ

In [2]:
def source(script, update=1):
    pipe = Popen(". %s; env" % script, stdout=PIPE, shell=True)
    data = pipe.communicate()[0]

    env = dict((line.split("=", 1) for line in data.splitlines()))
    if update:
        environ.update(env)

    return env

In [13]:
# If modifying these scopes, delete your previously saved credentials
# at ~/.credentials/sheets.googleapis.com-python-quickstart.json
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
CLIENT_SECRET_FILE = '../client_secret.json'
APPLICATION_NAME = 'Test'


def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        os.makedirs(credential_dir)
    credential_path = os.path.join(credential_dir,
                                   'sheets.googleapis.com-python-quickstart.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = APPLICATION_NAME
        if flags:
            credentials = tools.run_flow(flow, store, flags)
        else: # Needed only for compatibility with Python 2.6
            credentials = tools.run(flow, store)
        print 'Storing credentials to ' + credential_path 
    return credentials


"""Shows basic usage of the Sheets API.

ingrdients spreadsheet:
https://docs.google.com/spreadsheets/d/1LHojuRe1U7nhJUG-m7aMfA4BNQX-BmbmGlCSErE1NIg/edit
"""
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                'version=v4')
service = discovery.build('sheets', 'v4', http=http,
                          discoveryServiceUrl=discoveryUrl)

spreadsheetId = '1LHojuRe1U7nhJUG-m7aMfA4BNQX-BmbmGlCSErE1NIg'
rangeName = 'All_in_one'
result = service.spreadsheets().values().get(
    spreadsheetId=spreadsheetId, range=rangeName).execute()
values = result.get('values', [])

if not values:
    print 'No data found.'

scoodit_ingredients = pd.DataFrame(values[1:],columns=result.get('values')[0])
scoodit_ingredients = scoodit_ingredients.dropna(subset=['Categorie1'])
for col in scoodit_ingredients.columns:
    scoodit_ingredients[col] = scoodit_ingredients[col].str.lower().str.strip()
print 'list of scoodit ingredients from google doc'
scoodit_ingredients.head(60)

list of scoodit ingredients from google doc


,Group,Categorie1,Synonym1,Synonym2,Synonym3,Synonym4,Synonym5,wnid1,wnid2,wnid3
0,fruit,apple,,,,,,n07739125,None,None
1,fruit,apricot,,,,,,n07750872,None,None
2,fruit,banana,,,,,,n07753592,None,None
3,fruit,blackberry,,,,,,n07744811,None,None
4,fruit,blueberry,,,,,,n07743544,None,None
5,fruit,cantaloupe,,,,,,n07755929,None,None
6,fruit,cape gooseberry,,,,,,n12911079,None,None
7,fruit,carambola,star fruit,,,,,n07746551,None,None
8,fruit,cherimoya,,,,,,n07761141,None,None
9,fruit,cherry,,,,,,n07757132,None,None


In [14]:
_ = source('../scoodit.env')

downloader = imagedownloader.ImageNetDownloader()
username = os.environ['IMAGENET_USERNAME']
accessKey = os.environ['IMAGENET_ACCESSKEY']

userInfo = pref_utils.readUserInfo()
scoodit_wnids = {}
scoodit_ingredients.dropna(subset=['Categorie1','wnid1'],inplace=True)
wnid_cols = scoodit_ingredients.columns[scoodit_ingredients.columns.str.contains('wnid')]

In [15]:
os.getcwd()

'/home/kiril/PycharmProjects/scoodit_image_classification/notebooks'

In [32]:
scoodit_ingredients.iloc[140]

Group              fish
Categorie1         sole
Synonym1               
Synonym2               
Synonym3               
Synonym4               
Synonym5               
wnid1         n02664285
wnid2              None
wnid3              None
Name: 190, dtype: object

In [33]:
cwd=  os.getcwd()
for ind, ing in scoodit_ingredients.iloc[140:141].iterrows():
    for wnid in ing[wnid_cols].dropna():
        #download images
        desc = ing['Categorie1']
        #try:
        print wnid, desc
        downloader.download_original_images(wnid, username, accessKey,desc)
            #line = desc+';'+wnid+';'+'success\n'
            #with open(cwd+directory+filename, 'a') as f:
            #    f.write(line)
            #print line
        #except:
            #os.chdir(cwd)
            #with open(cwd+directory+filename, 'a') as f:
            #    f.write(desc+';'+wnid+';'+'failed\n')
            #print line

n02664285 sole
Downloading: http://www.image-net.org/download/synset?wnid=n02664285&username=nsckir&accesskey=293780e006d78276e0080b5fc6033cd45010671f&release=latest&src=stanford Bytes: 25722880
Extract images to /home/kiril/PycharmProjects/scoodit_image_classification/data/raw/sole_n02664285/n02664285_original_images


In [ ]:
veg_wnid = 'n07707451'
fruit_wnid = 'n07705931'
flavorer_wnid = 'n07809096'

print 'Vegetables Master Synset:','\t',veg_wnid
print 'Fruits Master Synset:','\t\t',fruit_wnid
print 'Flavorer Master Synset:','\t',flavorer_wnid

url1 = 'http://www.image-net.org/api/text/wordnet.structure.hyponym?wnid='
param1 = '&full=1'

response = urllib.urlopen(url1+veg_wnid+param1)
veg_wnids = pd.DataFrame(response.read().replace('-','').splitlines())
veg_wnids.columns = ['wnid']
veg_wnids['group']='vegetables'

response = urllib.urlopen(url1+fruit_wnid+param1)
fruit_wnids = pd.DataFrame(response.read().replace('-','').splitlines())
fruit_wnids.columns = ['wnid']
fruit_wnids['group']='fruits'

response = urllib.urlopen(url1+flavorer_wnid+param1)
flavorer_wnid = pd.DataFrame(response.read().replace('-','').splitlines())
flavorer_wnid.columns = ['wnid']
flavorer_wnid['group']='flavorers'

all_fr_veg = veg_wnids.append(fruit_wnids).append(flavorer_wnid)
print 'number of all edible fruits, vegetables and flavorers from imagenet',all_fr_veg.shape


l = scoodit_ingredients[wnid_cols].values
scoodit_wnids = set([item for sublist in l for item in sublist])
all_fr_veg_wnids = set(all_fr_veg['wnid'])

extra_veg_wnids = list(all_fr_veg_wnids-scoodit_wnids)
print 'number of extra edible fruits, vegetables and flavorers from imagenet',len(extra_veg_wnids)